In [1]:
import polars as pl

In [7]:
# 1. Load Data (as before)
metadata_pl = pl.read_csv("data/movies_metadata.csv", infer_schema_length=100000, n_rows=1000000)
ratings_pl = pl.read_csv("data/ratings.csv", infer_schema_length=100000, n_rows=1000000)

In [8]:
# 2. Process Metadata (Clean ID, Clean & Explode Genres)
metadata_processed_pl = (
    metadata_pl
    .lazy()
    .filter(pl.col("id").str.contains(r"^\d+$"))
    .with_columns(
        pl.col("id").cast(pl.Int64)
    )
    # --- Genre Cleaning and Exploding ---
    # 1. Filter out known bad strings (null, "", "[]")
    .filter(
        pl.col('genres').is_not_null()
        & (pl.col('genres') != "")
        & (pl.col('genres') != "[]")
    )
    .with_columns(
        # 2. Aggressive REGEX cleaning: Remove stray backslashes, convert single quotes to double quotes
        pl.col('genres')
          .str.replace_all(r"\\'", "'")  # Fix common escaped single quote
          .str.replace_all(r"\\", "")    # Remove all other stray backslashes
          .str.replace_all(r"'", '"')    # Convert all single quotes to double quotes (JSON compliant)
          .alias('genres_json'),
    )
    .with_columns(
        # 3. Parse the clean JSON string into a Polars Struct/List
        # The use of 'list' here tells Polars what we expect the output type to be.
        pl.col('genres_json').str.json_extract(pl.List(pl.Struct([pl.Field("id", pl.Int64), pl.Field("name", pl.Utf8)]))).alias('genres_struct'),
    )
    # 4. Filter out rows that failed JSON parsing
    .filter(
        pl.col('genres_struct').is_not_null()
    )
    .explode('genres_struct') # Explode the list of dictionaries
    .with_columns(
        # 5. Extract the 'name' field
        pl.col('genres_struct').struct.field('name').alias('genre')
    )
    # 6. Final filter to remove rows where the name extraction returned null
    .filter(
        pl.col('genre').is_not_null()
    )
    .select(
        pl.col("id"),
        pl.col("genre"),
        pl.col("original_language")
    )
    .collect()
)

In [9]:
# 3. Join Tables
movie_pl = metadata_processed_pl.join(ratings_pl, left_on="id", right_on='movieId', how='inner')

In [10]:
## Language Ratings
language_ratings = (
    movie_pl
    .group_by("original_language")
    .agg(
        pl.mean("rating").alias("avg_rating")
    )
    .sort("avg_rating", descending=True)
)
print("\n--- Average Rating by Original Language ---")
print(language_ratings)


--- Average Rating by Original Language ---
shape: (48, 2)
┌───────────────────┬────────────┐
│ original_language ┆ avg_rating │
│ ---               ┆ ---        │
│ str               ┆ f64        │
╞═══════════════════╪════════════╡
│ hr                ┆ 4.5        │
│ et                ┆ 4.219388   │
│ sk                ┆ 4.166667   │
│ ru                ┆ 4.006688   │
│ …                 ┆ …          │
│ te                ┆ 3.01676    │
│ cs                ┆ 2.878      │
│ tl                ┆ 2.75       │
│ null              ┆ 2.5        │
└───────────────────┴────────────┘


In [11]:
## Genre
genre_ratings = (
    movie_pl
    .group_by("genre")
    .agg(
        pl.mean("rating").alias("avg_rating")
    )
    .sort("avg_rating", descending=True)
)
print("--- Average Rating by Genre ---")
print(genre_ratings)

--- Average Rating by Genre ---
shape: (20, 2)
┌───────────┬────────────┐
│ genre     ┆ avg_rating │
│ ---       ┆ ---        │
│ str       ┆ f64        │
╞═══════════╪════════════╡
│ Western   ┆ 3.666209   │
│ Mystery   ┆ 3.664033   │
│ TV Movie  ┆ 3.626935   │
│ Animation ┆ 3.584484   │
│ …         ┆ …          │
│ Horror    ┆ 3.496328   │
│ Music     ┆ 3.452053   │
│ History   ┆ 3.428559   │
│ Family    ┆ 3.395616   │
└───────────┴────────────┘
